In [1]:
import tqdm
import torch
import wandb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

C:\Users\Sid\AppData\Local\Temp\ipykernel_14840\564061376.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
wandb.init(project="LoRA MobileBERT Model FineTuning Yo!")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddiqmoideen07. Use `wandb login --relogin` to force relogin


In [3]:
dataStuff = pd.read_csv("../augmentedDataStuff.csv")
dataStuff

,OCR Contents,Category
0,Invoice Number: INV0786Hotel Name: Spice Garde...,Food Category - Restaurant Bill/Receipt
1,Invoice No- DW/1112/851166\n\nInvoice Date : 2...,Travel Category - Flight Ticket
2,Hotel: Accor\r\nCity: Delhi\r\nNumber of Days:...,Miscellaneous Category - Miscellaneous Bills
3,FOOD INN FINE DINE\nProprietor\nPER =NNIA REST...,Food Category - Restaurant Bill/Receipt
4,= Confirmed Booking Id: NF20115276689422\n\nSU...,Travel Category - Flight Ticket
...,...,...
5161,Indiandit\n\nSERIAL SNA AUTONOBI\n\nr .A.B.Roa...,Fuel Category - Fuel Bill/Receipt
5162,Base Fare\nDistance\nTime\n\nSubtotal\n\nCHARG...,Travel Category - Cab Bill/Receipt
5163,"Dear sanjay kumar,\n\nCongratdatons! Thark you...",Travel Category - Train Ticket
5164,Invoice Number: INV0780Hotel Name: Taste of It...,Food Category - Restaurant Bill/Receipt


In [4]:
clsStuff = list(set(dataStuff["Category"]))
clsStuff

['Electronics Category - Electronic Gadgets Bill/Receipt',
 'Travel Category - Cab Bill/Receipt',
 'Travel Category - Flight Ticket',
 'Food Category - Restaurant Bill/Receipt',
 'Fuel Category - Fuel Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Room Stay Category - Stay Bill/Receipt',
 'Travel Category - Train Ticket',
 'Travel Category - Bus Ticket']

In [5]:
labelEncoder = LabelEncoder()
labelEncoder.fit(clsStuff)
clsDict = dict(zip(clsStuff, labelEncoder.transform(clsStuff)))
clsDict

{'Electronics Category - Electronic Gadgets Bill/Receipt': 0,
 'Travel Category - Cab Bill/Receipt': 6,
 'Travel Category - Flight Ticket': 7,
 'Food Category - Restaurant Bill/Receipt': 1,
 'Fuel Category - Fuel Bill/Receipt': 2,
 'Miscellaneous Category - Miscellaneous Bills': 3,
 'Room Stay Category - Stay Bill/Receipt': 4,
 'Travel Category - Train Ticket': 8,
 'Travel Category - Bus Ticket': 5}

In [6]:
numLabels = len(clsStuff)
batchSize = 16
learningRate = 2e-5
epochs = 450

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=numLabels).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/mobilebert-uncased")

c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, u

In [9]:
model.config

MobileBertConfig {
  "_name_or_path": "google/mobilebert-uncased",
  "architectures": [
    "MobileBertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_activation": false,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 512,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "intra_bottleneck_size": 128,
  "key_query_shared_bottleneck": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "mobilebert",
  "normalization_type": "no_norm",
  "num_attention_heads": 4,
  "num_f

In [10]:
loraConfig = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

In [11]:
model = get_peft_model(model, loraConfig)
model.print_trainable_parameters()

trainable params: 1,380,873 || all params: 25,967,378 || trainable%: 5.3177


In [12]:
wandb.watch(model)
wandb.config["learning_rate"] = learningRate
wandb.config["batch_size"] = batchSize

In [13]:
trainData, valData = train_test_split(dataStuff, test_size=0.2, random_state=42)
trainData['OCR Contents']

4711    Hotel: Hilton Worldwide\r\nCity: Amritsar\r\nN...
4184    goibibo sao\n\nFlight Ticket - Kolkata to Agar...
1670    Category: Utilities\r\nItem: Internet Bill\r\n...
99      Invoice Number: INV0810Hotel Name: The Hungry ...
3592    Invoice Number: INV0953Hotel Name: Gourmet Pal...
                              ...                        
4426    Invoice Number: INV0558Hotel Name: The Hungry ...
466     ticket ric 0: r07s220168 Gcleartrip\n\nBangalo...
3092    Category: Shopping\r\nItem: Accessories\r\nAmo...
3772    " ‘\n\nSREP iy\n\nCHOKHI DHANI PALACE HOTEL\n\...
860     Category: Healthcare\r\nItem: Medicines\r\nAmo...
Name: OCR Contents, Length: 4132, dtype: object

In [14]:
def prepData(data):
  train_inputs, train_masks, train_labels = [], [], []
  for item, label in zip(data["OCR Contents"], data["Category"]):
    text_content = item
    encoded_data = tokenizer(text_content, padding="max_length", truncation=True, max_length=512)
    train_inputs.append(encoded_data["input_ids"])
    train_masks.append(encoded_data["attention_mask"])
    train_labels.append(label)
  return train_inputs, train_masks, train_labels

In [15]:
def labels2Encoding(labelStuff):
    encodedLabels = labelEncoder.transform(labelStuff)
    return encodedLabels

In [16]:
trainInputs, trainMasks, trainLabels = prepData(trainData)
valInputs, valMasks, valLabels = prepData(valData)

In [17]:
trainLabels

['Miscellaneous Category - Miscellaneous Bills',
 'Travel Category - Bus Ticket',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Food Category - Restaurant Bill/Receipt',
 'Travel Category - Train Ticket',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Room Stay Category - Stay Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Miscellaneous Category - Miscellaneous Bills',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Travel Category - Bus Ticket',
 'Miscellaneous Category - Miscellaneous Bills',
 'Travel Category - Flight Ticket',
 'Food Category - Restaurant Bill/Receipt',
 'Food Category - Restaurant Bill/Receipt',
 'Travel Category - Train Ticket',
 'Travel Category - Bus Ticket',
 'T

In [18]:
trainLabels = labels2Encoding(trainLabels)
valLabels = labels2Encoding(valLabels)
trainLabels

array([3, 5, 3, ..., 3, 1, 3])

In [19]:
trainInputs = torch.tensor(trainInputs, dtype=torch.uint8).to(device)
trainMasks = torch.tensor(trainMasks, dtype=torch.uint8).to(device)
trainLabels = torch.tensor(trainLabels, dtype=torch.uint8).to(device)
valInputs = torch.tensor(valInputs, dtype=torch.uint8).to(device)
valMasks = torch.tensor(valMasks, dtype=torch.uint8).to(device)
valLabels = torch.tensor(valLabels, dtype=torch.uint8).to(device)

In [20]:
print(trainMasks.max(), trainInputs.min())

tensor(1, device='cuda:0', dtype=torch.uint8) tensor(0, device='cuda:0', dtype=torch.uint8)


In [21]:
trainDataset = TensorDataset(trainInputs, trainMasks, trainLabels)
trainDataloader = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)

In [22]:
valDataset = TensorDataset(valInputs, valMasks, valLabels)
valDataloader = DataLoader(valDataset, batch_size=batchSize)

In [23]:
optimizer = AdamW(model.parameters(), lr=learningRate)
criterion = torch.nn.CrossEntropyLoss()

c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
def train(model, optimizer, criterion, trainDataloader):
  model.train()
  total_loss = 0
  correct = 0
  for batch_idx, (input_ids, attention_mask, labels) in enumerate(trainDataloader):
    optimizer.zero_grad()
    input_ids = input_ids.long().to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    loss = criterion(outputs.logits, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    predictions = torch.argmax(outputs.logits, dim=1)
    correct += (predictions == labels).sum().item()
  return total_loss / len(trainDataloader), correct / len(trainDataset)

In [25]:
def validate(model, criterion, valDataloader):
  model.eval()
  total_loss = 0
  correct = 0
  all_preds, all_labels = [], []
  with torch.no_grad():
    for batch_idx, (input_ids, attention_mask, labels) in enumerate(valDataloader):
      input_ids = input_ids.long().to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      loss = criterion(outputs.logits, labels)
      total_loss += loss.item()
      predictions = torch.argmax(outputs.logits, dim=1)
      correct += (predictions == labels).sum().item()
      all_preds.extend(predictions.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())
  accuracy = accuracy_score(all_labels, all_preds)
  precision = precision_score(all_labels, all_preds, average="weighted")
  recall = recall_score(all_labels, all_preds, average="weighted")
  return total_loss / len(valDataloader), accuracy, precision, recall

In [26]:
for epoch in tqdm.trange(epochs, colour = "red", desc = "Epoch(s)"):
  train_loss, train_acc = train(model, optimizer, criterion, trainDataloader)
  val_loss, val_acc, val_precision, val_recall = validate(model, criterion, valDataloader)
  dataDict = {
    "Epoch": epoch+1,
    "Train Loss": train_loss,
    "Train Accuracy": train_acc,
    "Val Loss": val_loss,
    "Val Accuracy": val_acc,
    "Val Precision": val_precision,
    "Val Recall": val_recall
    }
  print(dataDict)
  wandb.log(dataDict)
  model.save_pretrained("tunedModels/trainedMobileBERTLoRA - Epoch {}".format(epoch))

Epoch(s):   0%|          | 0/450 [00:00<?, ?it/s]c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'Epoch': 1, 'Train Loss': 1134453.9004358712, 'Train Accuracy': 0.218054211035818, 'Val Loss': 22410.677103365386, 'Val Accuracy': 0.38878143133462284, 'Val Precision': 0.15115100135059806, 'Val Recall': 0.38878143133462284}


c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Epoch(s):   0%|          | 1/450 [01:52<14:04:34, 112.86s/it]c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'Epoch': 2, 'Train Loss': 15743.308936866553, 'Train Accuracy': 0.3228460793804453, 'Val Loss': 3593.375120192308, 'Val Accuracy': 0.38878143133462284, 'Val Precision': 0.15115100135059806, 'Val Recall': 0.38878143133462284}


c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Epoch(s):   0%|          | 2/450 [03:43<13:52:13, 111.46s/it]c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Epoch(s): 100%|█████████▉| 448/450 [13:46:31<03:41, 110.91s/it]

{'Epoch': 449, 'Train Loss': 0.07230082769248318, 'Train Accuracy': 0.9777347531461762, 'Val Loss': 0.6143522268359307, 'Val Accuracy': 0.8617021276595744, 'Val Precision': 0.8593956000647752, 'Val Recall': 0.8617021276595744}


c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Epoch(s): 100%|█████████▉| 449/450 [13:48:22<01:50, 110.79s/it]

{'Epoch': 450, 'Train Loss': 0.0631386178607626, 'Train Accuracy': 0.9818489835430784, 'Val Loss': 0.6652235529692776, 'Val Accuracy': 0.8597678916827853, 'Val Precision': 0.8610159331504561, 'Val Recall': 0.8597678916827853}


c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Epoch(s): 100%|██████████| 450/450 [13:50:13<00:00, 110.70s/it]


In [27]:
model.save_pretrained("trainedMobileBERTLoRA")